In [1]:
# from transformers import GPT2Config, TFGPT2Model
import tensorflow as tf
import numpy as np
import os

physical_devices = tf.config.list_physical_devices('GPU')
for physical_device in physical_devices:
  try:
    tf.config.experimental.set_memory_growth(physical_device, True)
  except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

import utils
import config

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
conf = config.Config("single_instruments_type", ROOT_PATH)

2022-11-23 21:22:56.216183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 21:22:56.922193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/lib64
2022-11-23 21:22:56.922238: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/l

In [2]:
# dataset = tf.data.Dataset.load(conf.lmda_genres_tf_data_path)   \
dataset = tf.data.Dataset.load(conf.tf_data_path)   \
    .shuffle(conf.SHUFFLE_SIZE)                                 \
    .batch(conf.BATCH_SIZE)                                     \
    .prefetch(conf.PREFETCH_SIZE)                               \
    .cache()                                                    


In [3]:
decoder_output = tf.random.uniform((conf.BATCH_SIZE, conf.SEQ_LEN, conf.TOKEN_DIM), minval=-1, maxval=1)

print(decoder_output.shape)

out_logits = [layer(decoder_output) for layer in conf.output_dense_layers]

for out_logit_part in out_logits:
    print(out_logit_part.shape)

for song, _ in dataset.take(1):
    pass

song= tf.slice(song, begin=[0, 0, 0], size = [-1,conf.SEQ_LEN-1,-1])

print("song_shape: {}".format(song.shape))


(2, 1024, 512)
(2, 1024, 8)
(2, 1024, 256)
(2, 1024, 131)
(2, 1024, 128)
(2, 1024, 136)
(2, 1024, 256)
(2, 1024, 129)
(2, 1024, 128)
(2, 1024, 25)
(2, 1024, 153)
(2, 1024, 49)
song_shape: (2, 1023, 11)


2022-11-23 21:22:59.428969: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [102]:
class MaskingActivationLayer(tf.keras.layers.Layer):

    def __init__(self, conf: config.Config, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
        
        self.conf = conf

        self.full_mask = conf.full_mask
        self.default_mask = conf.default_mask

        self.masked_activations = [tf.keras.layers.Softmax()]*len(conf.INPUT_RANGES)
    
        output_scores_dim = conf.SEQ_LEN-1
        vectors = [[1]*(i+1) + [0]*(output_scores_dim-i-1) for i in range(output_scores_dim)]
        self.triang_mask = tf.cast(tf.repeat(tf.expand_dims(np.asarray(np.stack(vectors)), axis=-1), repeats=[11], axis=-1), dtype=tf.dtypes.float32)

    
    def softargmax(x, beta=1e10):
        x = tf.convert_to_tensor(x, dtype=tf.float64)
        x_range = tf.range(x.shape.as_list()[-1], dtype=x.dtype)
        return tf.reduce_sum(tf.nn.softmax(x*beta) * x_range, axis=-1)

    
    def create_manual_mask(up_to, size):
        
        return 


    def get_mask(self, inputs):

        chosen_type, song, scores, index_tensor = inputs
    
        chosen_type = tf.cast(chosen_type, dtype=tf.dtypes.int64)               # 1
        song = tf.cast(song, dtype=tf.dtypes.int64)                             # (SEQ_LEN-1) * 11
        scores = tf.cast(scores, dtype=tf.dtypes.int64)                         # (SEQ_LEN-1)
        index_tensor = tf.squeeze(tf.cast(index_tensor, dtype=tf.dtypes.int64)) # 1

        default_token_parts = [True, True, True, True, True, True, True, True, True, True, True]
        default_flag = False

        # DEFAULT CREATION (tensorflow requires the variable to be present in all possible branches when if/else are present)
        min_measure =           tf.constant(-1, dtype=tf.dtypes.int64)
        min_beat =              tf.constant(-1, dtype=tf.dtypes.int64)
        min_position =          tf.constant(-1, dtype=tf.dtypes.int64)
        note = False
        allowed_instruments = tf.constant([0]*conf.INPUT_RANGES["instrument"], dtype=tf.dtypes.int64)
        allowed_key_sign =      tf.constant(-1, dtype=tf.dtypes.int64)
        allowed_time_sign =     tf.constant(-1, dtype=tf.dtypes.int64)
        allowed_tempo =         tf.constant(-1, dtype=tf.dtypes.int64)
        forbidden_instruments = tf.constant([0]*conf.INPUT_RANGES["instrument"], dtype=tf.dtypes.int64)
        forbidden_key_sign =    tf.constant(-1, dtype=tf.dtypes.int64)
        forbidden_time_sign =   tf.constant(-1, dtype=tf.dtypes.int64)
        forbidden_tempo =       tf.constant(-1, dtype=tf.dtypes.int64)

        forbidden_instruments_flag = False
        
        if chosen_type == 0: # TODO: change to 7 # only way it chooses 0 is that max_type==7 --> AFTER END OF SONG --> only thing the model can do is guess all zeros
            # "does not have to learn nothing" --> it's all zeros just like the padding tensors
            default_token_parts = [False, True, True, True, True, True, True, True, True, True, True]
            default_flag = True

        # instrument selection
        elif chosen_type == 1: # false only for type and instrument type (the ones that you can choose)
            if tf.cast(tf.gather_nd(song, indices=[[index_tensor,0]]), tf.dtypes.int64) == 0:
                # first instrument chosen
                default_token_parts = [False, True, True, True, True, True, True, True, True, True, True]
                default_flag = True
                
            else:
                # from second instrument chosen onwards this is right
                forbidden_instruments, _ = tf.unique(tf.gather(
                        tf.squeeze(tf.slice(song, begin=[0,6], size=[-1,1]), axis=-1), 
                        tf.squeeze(tf.where(tf.squeeze(tf.slice(song, begin=[0,0], size=[-1,1])) == 2))
                ), out_idx=tf.dtypes.int64)

                forbidden_instruments_flag = True
            
        
        # start of events
        elif chosen_type == 2: # false only for type (cannot choose anything in "start of events" token)
            default_token_parts = [False, True, True, True, True, True, True, True, True, True, True]
            default_flag = True
        
        # notes
        elif chosen_type == 3: # note: has same key_sign, time_sign and tempo as last previous event, everything has to be manually decided
            
            min_measure = tf.cast(tf.gather_nd(song, indices=[[index_tensor,1]]), tf.dtypes.int64)   # it has to be >= than the last measure

            # if in the MEASURE SCORES the MAX SCORE between all possible measures >= min_measure is the one == to min_measure
            # it means that the model will choose the same measure as last event --> we need to make sure that beat >= last_beat
            if tf.math.argmax(tf.slice(
                    scores, 
                    begin=conf.INPUT_RANGES["type"] + min_measure, 
                    size=conf.INPUT_RANGES["measure"]-min_measure
                )) == min_measure:
                
                min_beat = tf.squeeze(tf.cast(tf.gather_nd(song, indices=[[index_tensor,2]]), tf.dtypes.int64), axis=0)      # it has to be >= than the last beat (if measure is the same)

                if tf.math.argmax(tf.slice(
                        scores, 
                        begin=[conf.INPUT_RANGES["type"]+conf.INPUT_RANGES["measure"]+min_beat], 
                        size=[conf.INPUT_RANGES["beat"]-min_beat]
                    )) == min_beat:

                    min_position = tf.squeeze(tf.cast(tf.gather_nd(song, indices=[[index_tensor,3]]), tf.dtypes.int64), axis=0)  # it has to be >= than the last position (if beat and measure are the same)
                else:
                    min_position = tf.constant(0, dtype=tf.dtypes.int64)
            else:
                min_beat = tf.constant(0, dtype=tf.dtypes.int64)
                min_position = tf.constant(0, dtype=tf.dtypes.int64)
            
            note = True

            # complicated but --> gather the values of token_part 6 corresponding to the token indices where token_part == 2 (+ unique for no repetition)

            allowed_instruments, _ = tf.unique(tf.gather(
                    tf.squeeze(tf.slice(song, begin=[0,6], size=[-1,1]), axis=-1), 
                    tf.squeeze(tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 2))
            ), out_idx=tf.dtypes.int64)

            allowed_key_sign = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,8], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 4)[-1] # if type == 4 --> read the LAST key_sign
            ), dtype=tf.dtypes.int64)
            allowed_time_sign = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,9], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 5)[-1] # if type == 5 --> read the LAST time_sign
            ), dtype=tf.dtypes.int64)
            allowed_tempo = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,10], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 6)[-1] # if type == 6 --> read the LAST tempo
            ), dtype=tf.dtypes.int64)
        
        # key_sign, time_sign, tempo
        elif chosen_type >= 4 and chosen_type <= 6:

            # if last event is at the beginning of a measure, you can add an event at the same time
            if tf.gather_nd(song, indices=[[index_tensor,3]]) == 0 and tf.gather_nd(song, indices=[[index_tensor,2]]) == 0:  # if beat and position == 0, can be this measure
                min_measure = tf.squeeze(tf.cast(tf.gather_nd(song, indices=[[index_tensor,1]])   , tf.dtypes.int64), axis=0)
            # otherwise it goes to the next measure
            else:
                min_measure = tf.squeeze(tf.cast(tf.gather_nd(song, indices=[[index_tensor,1]]) +1, tf.dtypes.int64), axis=0)
            
            if chosen_type == 4:
                
                forbidden_key_sign = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,8], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 4)[-1] # if type == 4 --> read the LAST key_sign
                ), dtype=tf.dtypes.int64) # cannot put the same key_sign again

            elif chosen_type == 5:

                forbidden_time_sign = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,9], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 5)[-1] # if type == 5 --> read the LAST time_sign
                ), dtype=tf.dtypes.int64) # cannot put the same time_sign again
            
            elif chosen_type == 6:

                forbidden_tempo = tf.cast(tf.gather(
                    tf.slice(song, begin=[0,10], size=[-1,1]), 
                    tf.where(tf.slice(song, begin=[0,0], size=[-1,1]) == 6)[-1] # if type == 6 --> read the LAST tempo
                ), dtype=tf.dtypes.int64) # cannot put the same tempo again
            
            else:
                # cannot RAISE inside tf graph, because it WILL pass from every path
                default_token_parts = [True, True, True, True, True, True, True, True, True, True, True]
                default_flag = True
                # raise ValueError("Impossible that chosen type isn't in [4,6] inside here --> {}".format(chosen_type))

        elif chosen_type == 7: # end of song --> only type can be chosen, all the others are default
            default_token_parts = [False, True, True, True, True, True, True, True, True, True, True]
            default_flag = True

        else:
            default_token_parts = [True, True, True, True, True, True, True, True, True, True, True]
            default_flag = True
            # raise ValueError("Impossible that chosen type isn't in [1,7] --> {}".format(chosen_type))

        measure_mask =      self.default_mask[1]
        beat_mask =         self.default_mask[2]
        position_mask =     self.default_mask[3]
        duration_mask =     self.default_mask[4]
        pitch_mask =        self.default_mask[5]
        instruments_mask =  self.default_mask[6]
        velocity_mask =     self.default_mask[7]
        key_sign_mask =     self.default_mask[8]
        time_sign_mask =    self.default_mask[9]
        tempo_mask =        self.default_mask[10]

            
        if default_flag: # no manual masking, either "can freely choose this part of the token" or "can only choose default for this part of the token"

            return tf.cast(tf.concat(
                        [self.default_mask[i] if default_token_parts[i] else self.full_mask[i] for i in range(len(default_token_parts))],
                        axis=-1
                    ), dtype=tf.dtypes.float32)
        
        else: # manual masking

            if not forbidden_instruments_flag:
                # measure mask, beat and position go to default if type==2 and forbidden_instruments_flag == True
                # so if forbidden_instruments_flag == False --> you can change it
                measure_mask = tf.convert_to_tensor([[False]*min_measure + [True]*(conf.INPUT_RANGES["measure"]-min_measure)], dtype=tf.dtypes.bool)

                if min_beat != tf.constant(-1, dtype=tf.dtypes.int64):
                    # oss: allowed_time_sign is always != None if min_beat != None
                    max_beat = conf.get_max_beat_from_time_sign(allowed_time_sign)
                    
                    # allowed beats are only AFTER previous beat and BEFORE max_beat from the numerator of the time_sign
                    beat_mask = tf.convert_to_tensor([
                        [False]*min_beat + \
                        [True]*(max_beat-min_beat) + \
                        [False]*(conf.INPUT_RANGES["beat"]-max_beat)
                    ], dtype=tf.dtypes.bool)

                if min_position != tf.constant(-1, dtype=tf.dtypes.int64):
                    position_mask = tf.convert_to_tensor([[False]*min_position + [True]*(conf.INPUT_RANGES["position"]-min_position)], dtype=tf.dtypes.bool)

            else:
                instruments_mask = tf.sparse.SparseTensor(#forbidden
                    indices=tf.expand_dims(forbidden_instruments,axis=-1),
                    values=tf.squeeze(tf.zeros_like(forbidden_instruments)),
                    dense_shape=[conf.INPUT_RANGES["instrument"]]
                )

                instruments_mask = tf.cast(tf.sparse.to_dense(instruments_mask, default_value=1), dtype=tf.dtypes.bool)
                # FORBIDDEN INSTRUMENTS is ONLY USED WHEN type==1 --> measure_mask, beat, position are all default

            if chosen_type==3:

                duration_mask = self.full_mask[4]
                pitch_mask = self.full_mask[5]
                velocity_mask = self.full_mask[7]

                instruments_mask = tf.sparse.SparseTensor(#allowed
                    indices=tf.stack([
                        allowed_instruments,
                        tf.zeros_like(allowed_instruments)
                    ]),
                    values=tf.ones_like(allowed_instruments),
                    dense_shape=[conf.INPUT_RANGES["instrument"]]
                )

                instruments_mask = tf.cast(tf.sparse.to_dense(instruments_mask, default_value=0), dtype=tf.dtypes.bool)

                
            if allowed_key_sign != tf.constant(-1, dtype=tf.dtypes.int64):
                key_sign_mask = tf.convert_to_tensor([[True if i == allowed_key_sign else False for i in range(conf.INPUT_RANGES["key_sign"])]], dtype=tf.dtypes.bool)
            else:
                if forbidden_key_sign != tf.constant(-1, dtype=tf.dtypes.int64):
                    key_sign_mask = tf.convert_to_tensor([[False if i == forbidden_key_sign else True for i in range(conf.INPUT_RANGES["key_sign"])]], dtype=tf.dtypes.bool)
                else:
                    pass
                    # raise AssertionError("Cannot have both allowed and forbidden key_sign not instanciated")

            if allowed_time_sign != tf.constant(-1, dtype=tf.dtypes.int64):
                time_sign_mask = tf.convert_to_tensor([[True if i == allowed_time_sign else False for i in range(conf.INPUT_RANGES["time_sign"])]], dtype=tf.dtypes.bool)
            else:
                if forbidden_time_sign != tf.constant(-1, dtype=tf.dtypes.int64):
                    time_sign_mask = tf.convert_to_tensor([[False if i == forbidden_time_sign else True for i in range(conf.INPUT_RANGES["time_sign"])]], dtype=tf.dtypes.bool)
                else:
                    pass
                    # raise AssertionError("Cannot have both allowed and forbidden time_sign not instanciated")

            if allowed_tempo != tf.constant(-1, dtype=tf.dtypes.int64):
                tempo_mask = tf.convert_to_tensor([[True if i == allowed_tempo else False for i in range(conf.INPUT_RANGES["tempo"])]], dtype=tf.dtypes.bool)
            else:
                if forbidden_tempo != tf.constant(-1, dtype=tf.dtypes.int64):
                    tempo_mask = tf.convert_to_tensor([[False if i == forbidden_tempo else True for i in range(conf.INPUT_RANGES["tempo"])]], dtype=tf.dtypes.bool)
                else:
                    pass
                    # raise AssertionError("Cannot have both allowed and forbidden tempo not instanciated")

            return tf.cast(tf.expand_dims(tf.concat([
                tf.squeeze(self.full_mask[0]), # type is not masked
                tf.squeeze(measure_mask),
                tf.squeeze(beat_mask),
                tf.squeeze(position_mask),
                tf.squeeze(duration_mask),
                tf.squeeze(pitch_mask),
                tf.squeeze(instruments_mask),
                tf.squeeze(velocity_mask),
                tf.squeeze(key_sign_mask),
                tf.squeeze(time_sign_mask),
                tf.squeeze(tempo_mask)
            ], axis=-1), axis=0), dtype=tf.dtypes.float32)
            

    def get_mask_for_all_tokens(self, inputs): 

        '''
        Returns a list of ndarrays of bool type used for masking

        inputs are for a SINGLE ELEMENT OF A BATCH of size SEQ_LEN*(1+11+1399) where 1399 is the summed length of logits
        '''
        # defaults (tf graph needs them with if/elses)

        chosen_type_each_token = tf.cast(tf.squeeze(tf.slice(inputs, begin=[0,0], size=[-1,1]), axis=-1), dtype=tf.dtypes.int64)
        song = tf.cast(tf.slice(inputs, begin=[0,1], size=[-1,11]), dtype=tf.dtypes.float32)
        scores = tf.slice(inputs, begin=[0,1+11], size=[-1,-1])

        # result = 6143 matrices --> first one has only first token full (others are 0), second one has first two tokens full (others are 0) etc
        triang_song = tf.math.multiply(
            tf.repeat(tf.expand_dims(song, axis=0), repeats=[conf.SEQ_LEN-1], axis=0),
            self.triang_mask
        )

        index_tensor = tf.expand_dims(tf.constant([i for i in range(conf.SEQ_LEN-1)], dtype=tf.dtypes.int64), axis=-1)
        
        # TODO: to speedup we could change it to vectorized_map, but we need to try it out
        # map_fn is stupid and wants float32 (i think) both in input and output
        final_masks = tf.map_fn(
            fn=self.get_mask, 
            elems=(
                tf.cast(chosen_type_each_token, dtype=tf.dtypes.float32),   # (SEQ_LEN-1)*1                 --> 1
                triang_song,                                                # (SEQ_LEN-1)*(SEQ_LEN-1)*11    --> (SEQ_LEN-1)*11
                scores,                                                     # (SEQ_LEN-1)*1399              --> 1399
                tf.cast(index_tensor, dtype=tf.dtypes.float32)              # (SEQ_LEN-1)*1                 --> 1
            ), 
            fn_output_signature=tf.dtypes.float32
        ) # TODO: change it accordingly to the output signature

        return final_masks
            
        


    def call(self, song, out_logits:list):
        '''
        Takes as input:
            - out_logits: the scores outputted by the decoder + dense layers
            - song: the input song, token by token

        This function, based on the chosen token "type" given by the first dense layer,
        masks all the different parts of the token accordingly, also taking into consideration
        the previous tokens of the song

        Output:
            - probabilities for each different part of the predicted token (the following one in the song)
        
        '''
        concat_logits = tf.concat(out_logits, axis = -1)

        max_type = tf.math.reduce_max(song[:,0])

        # do not have to be tensors because loos shouldn't flow through them
        if max_type == 0: # only start of song token
            # cannot be anything else than instrument choice (1)
            type_mask = np.asarray([False, True, False, False, False, False, False, False], dtype=bool)
        elif max_type == 1: # we reached instrument choice
            # cannot be anything else than instrument choice (1) or start of events (2)
            type_mask = np.asarray([False, True, True, False, False, False, False, False], dtype=bool)
        elif max_type >= 2 and max_type < 7: # we reached start of events or notes
            type_mask = np.asarray([False, False, False, True, True, True, True, True], dtype=bool)

        elif max_type == 7: # at the end of the song we can ONLY GUESS "000000000" TODO: change to zero
            type_mask = np.asarray([True, False, False, False, False, False, False, False], dtype=bool)

        else:
            type_mask = np.asarray([False, False, False, False, False, False, False, False], dtype=bool)
            # raise ValueError("Impossible that max_type is not in [0,7]: {}".format(max_type))

        type_scores = self.masked_activations[0](out_logits[0], type_mask) # the first masked activation is for the type
        
        # needs to be differentiable (but the masks shouldn't need to be)
        # chosen_type = self.softargmax(type_scores)
        chosen_type = tf.expand_dims(tf.math.argmax(type_scores, axis=-1, output_type=tf.dtypes.uint16)[:,1:], axis=-1)

        # could change == i with x<i+eps and x>i-eps because it's softargmax and not argmax

        # map_fn always unpacks axis=0      
        # TODO: COULD TUPLE INSTEAD OF CONCAT  
        masks = tf.map_fn(self.get_mask_for_all_tokens, tf.concat([
                tf.cast(chosen_type, tf.dtypes.float32),                # BATCH*(SEQ_LEN-1)
                tf.cast(song, tf.dtypes.float32),                       # BATCH*(SEQ_LEN-1)*11
                tf.slice(concat_logits, begin=[0,0,0], size=[-1,conf.SEQ_LEN-1,-1]) # BATCH*(SEQ_LEN-1)*1399
            ], axis = -1))

        return masks
        return [masked_act(type_logit, type_mask) for masked_act, type_logit, type_mask in zip(self.masked_activations, out_logits, mask)]


# do a simple model with "out_logits" dimension in input and see only 

In [103]:
song_input = tf.keras.layers.Input(tensor=song)
out_logits_input = [tf.keras.layers.Input(tensor=out_logit_part) for out_logit_part in out_logits]

y = MaskingActivationLayer(conf)(song_input, out_logits_input)

model = tf.keras.Model([song_input, out_logits_input], y)

ValueError: Exception encountered when calling layer "masking_activation_layer_41" (type MaskingActivationLayer).

in user code:

    File "/tmp/ipykernel_113789/2581647075.py", line 374, in call  *
        masks = tf.map_fn(self.get_mask_for_all_tokens, tf.concat([
    File "/tmp/ipykernel_113789/2015448542.py", line 313, in get_mask_for_all_tokens  *
        final_masks = tf.map_fn(
    File "/tmp/ipykernel_113789/3558827725.py", line 71, in get_mask  *
        forbidden_instruments, _ = tf.unique(tf.gather_nd(

    ValueError: Shape must be rank 1 but is rank 3 for '{{node masking_activation_layer_41/map/while/map/while/cond/cond/cond/Unique}} = Unique[T=DT_INT64, out_idx=DT_INT64](masking_activation_layer_41/map/while/map/while/cond/cond/cond/GatherNd)' with input shapes: [1,2,11].


Call arguments received by layer "masking_activation_layer_41" (type MaskingActivationLayer):
  • song=tf.Tensor(shape=(2, 1023, 11), dtype=uint8)
  • out_logits=['tf.Tensor(shape=(2, 1024, 8), dtype=f

In [ ]:
model([song, out_logits])

InvalidArgumentError: Exception encountered when calling layer 'masking_activation_layer_40' (type MaskingActivationLayer).

in user code:

    File "/tmp/ipykernel_113789/2015448542.py", line 313, in get_mask_for_all_tokens  *
        final_masks = tf.map_fn(
    File "/tmp/ipykernel_113789/2428752462.py", line 71, in get_mask  *
        forbidden_instruments, _ = tf.unique(tf.gather_nd(

    InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [0] != values[1].shape = [] [Op:Pack] name: 0


Call arguments received by layer 'masking_activation_layer_40' (type MaskingActivationLayer):
  • song=tf.Tensor(shape=(2, 1023, 11), dtype=uint8)
  • out_logits=['tf.Tensor(shape=(2, 1024, 8), dtype=float32)', 'tf.Tensor(shape=(2, 1024, 256), dtype=float32)', 'tf.Tensor(shape=(2, 1024, 131), dtype=float32)', 'tf.Tensor(shape=(2, 1024, 128), dtype=float32)', 'tf.Tensor(shape=(2, 1024, 136)

In [6]:
class MusicGenerator(tf.keras.Model):

    def __init__(self, conf: config.Config, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.conf = conf
        self.SEQ_LEN = conf.SEQ_LEN
        self.TOKEN_DIM = conf.TOKEN_DIM
        self.INPUT_RANGES = conf.INPUT_RANGES

        self.dense_genre_emb = tf.keras.layers.Dense(self.TOKEN_DIM)
        self.embeddings = conf.embedding_layers
        self.concat_layer = tf.keras.layers.Concatenate(axis=2)

        self.pos_embedding_matrix = conf.get_positional_embedding_matrix()
        self.positional_embeddings = tf.stack([self.pos_embedding_matrix]*conf.BATCH_SIZE)
        self.sum_layer = tf.keras.layers.Add()

        # TODO: add dense layer from embeddings to input decoder

        self.decoder = conf.get_decoder()

        self.output_dense_layers = conf.output_dense_layers

        self.full_mask = conf.full_mask
        self.default_mask = conf.default_mask

        self.masked_activations = [tf.keras.layers.Softmax()]*len(self.embeddings)


    def softargmax(x, beta=1e10):
        x = tf.convert_to_tensor(x, dtype=tf.float64)
        x_range = tf.range(x.shape.as_list()[-1], dtype=x.dtype)
        return tf.reduce_sum(tf.nn.softmax(x*beta) * x_range, axis=-1)


    def apply_activations(self, logits, masks):
        return [activation(elem, mask) for elem, mask, activation in zip(logits, masks, self.masked_activations)]


    def get_mask(self, 
            default = [], 
            min_measure = None, 
            min_beat = None, 
            min_position = None,
            note = False,
            allowed_instruments = None,
            allowed_key_sign = None,
            allowed_time_sign = None,
            allowed_tempo = None,
            forbidden_key_sign = None,
            forbidden_time_sign = None,
            forbidden_tempo = None
        ): 

        '''
        Returns a list of ndarrays of bool type used for masking
        '''
        if len(default) > 0: # no manual masking, either "can freely choose this part of the token" or "can only choose default for this part of the token"
            return [self.default_mask[i] if default[i] else self.full_mask[i] for i in range(len(default))]
        
        else: # manual masking

            measure_mask = np.asarray([False]*min_measure + [True]*(self.INPUT_RANGES["measure"]-min_measure), dtype=bool)
            # TODO: Implement BEAT MASK only if measure == last_measure
            # TODO: Implement POSITION MASK only if measure == last_measure AND beat == last_beat

            if min_beat == None:
                beat_mask = self.default_mask[2]
            else: # oss: allowed_time_sign is always != None if min_beat != None
                max_beat = conf.get_max_beat_from_time_sign(allowed_time_sign)
                # allowed beats are only AFTER previous beat and BEFORE max_beat from the numerator of the time_sign
                beat_mask = np.asarray(
                    [False]*min_beat + \
                    [True]*(max_beat-min_beat) + \
                    [False]*(self.INPUT_RANGES["beat"]-max_beat)
                , dtype=bool)
            

            if min_position == None:
                position_mask = self.default_mask[3]
            else:
                position_mask = np.asarray([False]*min_position + [True]*(self.INPUT_RANGES["position"]-min_position), dtype=bool)

            if note:
                duration_mask = self.full_mask[4]
                pitch_mask = self.full_mask[5]
                velocity_mask = self.full_mask[7]
            else:
                duration_mask = self.default_mask[4]
                pitch_mask = self.default_mask[5]
                velocity_mask = self.default_mask[7]

            if allowed_instruments == None:
                instruments_mask = self.default_mask[6]
            else:
                instruments_mask = np.asarray([True if i in allowed_instruments else False for i in range(self.INPUT_RANGES["instrument"])], dtype=bool)

            if allowed_key_sign == None:
                if forbidden_key_sign == None:
                    raise AssertionError("Cannot have both allowed and forbidden key_sign not instanciated")
                else:
                    key_sign_mask = np.asarray([False if i == forbidden_key_sign else True for i in range(self.INPUT_RANGES["key_sign"])], dtype=bool)
            else:
                key_sign_mask = np.asarray([True if i == allowed_key_sign else False for i in range(self.INPUT_RANGES["key_sign"])], dtype=bool)

            if allowed_time_sign == None:
                if forbidden_time_sign == None:
                    raise AssertionError("Cannot have both allowed and forbidden time_sign not instanciated")
                else:
                    time_sign_mask = np.asarray([False if i == forbidden_time_sign else True for i in range(self.INPUT_RANGES["time_sign"])], dtype=bool)
            else:
                time_sign_mask = np.asarray([True if i == allowed_time_sign else False for i in range(self.INPUT_RANGES["time_sign"])], dtype=bool)

            if allowed_tempo == None:
                if forbidden_tempo == None:
                    raise AssertionError("Cannot have both allowed and forbidden tempo not instanciated")
                else:
                    tempo_mask = np.asarray([False if i == forbidden_tempo else True for i in range(self.INPUT_RANGES["tempo"])], dtype=bool)
            else:
                tempo_mask = np.asarray([True if i == allowed_tempo else False for i in range(self.INPUT_RANGES["tempo"])], dtype=bool)


            return [
                self.full_mask[0], # type is not masked
                measure_mask,
                beat_mask,
                position_mask,
                duration_mask,
                pitch_mask,
                instruments_mask,
                velocity_mask,
                key_sign_mask,
                time_sign_mask,
                tempo_mask
            ]


    def mask_and_activate_outputs(self, out_logits, song):
        '''
        Takes as input:
            - out_logits: the scores outputted by the decoder + dense layers
            - song: the input song, token by token

        This function, based on the chosen token "type" given by the first dense layer,
        masks all the different parts of the token accordingly, also taking into consideration
        the previous tokens of the song

        Output:
            - probabilities for each different part of the predicted token (the following one in the song)
        
        '''

        max_type = tf.math.reduce_max(song[:,0])

        # do not have to be tensors because loos shouldn't flow through them
        if max_type == 0: # only start of song token
            # cannot be anything else than instrument choice (1)
            type_mask = np.asarray([False, True, False, False, False, False, False, False], dtype=bool)
        elif max_type == 1: # we reached instrument choice
            # cannot be anything else than instrument choice (1) or start of events (2)
            type_mask = np.asarray([False, True, True, False, False, False, False, False], dtype=bool)
        elif max_type >= 2 and max_type < 7: # we reached start of events or notes
            type_mask = np.asarray([False, False, False, True, True, True, True, True], dtype=bool)

        elif max_type == 7: # at the end of the song we can ONLY GUESS "000000000" TODO: change to zero
            type_mask = np.asarray([True, False, False, False, False, False, False, False], dtype=bool)

        type_scores = self.masked_activations[0](out_logits[0], type_mask) # the first masked activation is for the type
        
        # needs to be differentiable (but the masks shouldn't need to be)
        chosen_type = self.softargmax(type_scores)

        # could change == i with x<i+eps and x>i-eps because it's softargmax and not argmax
        if chosen_type == 0: # TODO: change to 7 # only way it chooses 0 is that max_type==7 --> AFTER END OF SONG --> only thing the model can do is guess all zeros
            # "does not have to learn nothing" --> it's all zeros just like the padding tensors
            mask = self.get_mask(default = [False, True, True, True, True, True, True, True, True, True, True])
        # instrument selection
        if chosen_type == 1: # false only for type and instrument type (the ones that you can choose)
            mask = self.get_mask(default = [False, True, True, True, True, True, False, True, True, True, True])
        # start of events
        elif chosen_type == 2: # false only for type (cannot choose anything in "start of events" token)
            mask = self.get_mask(default = [False, True, True, True, True, True, True, True, True, True, True])
        # notes
        elif chosen_type == 3: # note: has same key_sign, time_sign and tempo as last previous event, everything has to be manually decided
            
            mask = self.get_mask(
                min_measure = song[-1,1],   # it has to be >= than the last measure
                min_beat = song[-1,2],      # it has to be >= than the last beat (if measure is the same)
                min_position = song[-1,3],  # it has to be >= than the last position (if beat and measure are the same)
                note = True,
                allowed_instruments = np.unique(song[np.where(song[:,0] == 2),6]), # if type == 2 --> read the instruments (unique = set)
                allowed_key_sign =  song[np.where(song[:,0] == 4), 8][0][-1], # if type == 4 --> read the LAST key_sign
                allowed_time_sign = song[np.where(song[:,0] == 5), 9][0][-1], # if type == 5 --> read the LAST time_sign
                allowed_tempo =     song[np.where(song[:,0] == 6),10][0][-1]  # if type == 6 --> read the LAST tempo
            )

        # key_sign, time_sign, tempo
        elif chosen_type == 4 or chosen_type == 5 or chosen_type == 6:
            # if last event is at the beginning of a measure, you can add an event at the same time
            if song[-1,3] == 0 and song[-1,2] == 0:  # if beat and position == 0, can be this measure
                min_measure = song[-1,1]
            # otherwise it goes to the next measure
            else:
                min_measure = song[-1,1] + 1
            
            if chosen_type == 4:
                mask = self.get_mask(
                    min_measure = min_measure,
                    forbidden_key_sign = song[np.where(song[:,0] == 4), 8][0][-1] # cannot put the same key_sign again
                )

            if chosen_type == 5:
                mask = self.get_mask(
                    min_measure = min_measure,
                    forbidden_time_sign = song[np.where(song[:,0] == 5), 9][0][-1] # cannot put the same time_sign again
                )
            
            if chosen_type == 6:
                mask = self.get_mask(
                    min_measure = min_measure,
                    forbidden_tempo = song[np.where(song[:,0] == 6),10][0][-1] # cannot put the same tempo again
                )
        
        elif chosen_type == 7: # end of song --> only type can be chosen, all the others are default
            mask = self.get_mask(default = [False, True, True, True, True, True, True, True, True, True, True])

        else:
            raise ValueError("Impossible that chosen type isn't in [1,7] --> {}".format(chosen_type))

        # TODO: check if it works for every type!
        return [masked_act(type_logit, type_mask) for masked_act, type_logit, type_mask in zip(self.masked_activations, out_logits, mask)]


    def call(self, inputs):

        # to train you need to add a "end_song" input --> in this way you create the attention mask
        # for the decoder

        if type(inputs) == dict:
            song = inputs["song"]
            genre = inputs["genre"]
            attention_mask = inputs["attention_mask"] # TODO: remove if decoder outputs sequences

        elif type(inputs) == tuple:
            song = inputs[0]
            genre = inputs[1]
            attention_mask = inputs[2] # TODO: remove if decoder outputs sequences
            
        # EMBEDDING GENERATION for decoder
        genre_embedding = self.dense_genre_emb(genre)

        # TODO: check how they come out (should be 11*64 numbers for each line, and 6143 lines for each song in batch)
        # TODO: batch?
        token_embeddings = [self.embeddings[i](song[:,i]) for i in range(len(self.embeddings))]

        final_embeddings = self.concat_layer([genre_embedding, token_embeddings])

        decoder_output = self.decoder(
            input_embeds = final_embeddings,
            attention_mask = attention_mask, # TODO: remove if decoder outputs sequences
            position_ids = self.positional_embeddings
        )

        out_logits = [layer(decoder_output["last_hidden_state"]) for layer in self.output_dense_layers]
        
        # insert for if decoder outputs sequences

        out_scores = self.mask_and_activate_outputs(out_logits, song)
        
        # return genre_embedding, token_embeddings, final_embeddings, attention_mask, out_logits, out_scores
        return out_scores

    
    def train_step(self, data):
        song, genre = data

        with tf.GradientTape() as tape:
            # TODO: how to do it for batches?
            trainable_vars = self.trainable_variables
            # we only want to predict up to the real finish of the song
            # but they are padded --> for each song we stop at the token BEFORE the "end_song" token
            # we predict as last token the "end_song" one

            # pass through the network THE ENTIRE SONG (even the last PADDED TOKENS! so the batch flows together)
            # OR we could stop on the biggest last token in batch
            for i, y in enumerate(song[1:]):
                # TODO: check and simplify if decoder output is already sequence

                # y is the current token
                y_pred = self((
                    song,
                    genre,
                    np.asarray([1]*(i+1) + [0]*(self.SEQ_LEN-1-(i+1))) # attention mask
                ))

                loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
                gradients = tape.gradient(loss, trainable_vars)

                self.optimizer.apply_gradients(zip(gradients, trainable_vars))
                self.compiled_metrics.update_state(y, y_pred)
        
        return {m.name: m.result() for m in self.metrics}







TypeError: 'type' object is not subscriptable

In [14]:
model = MusicGenerator(conf)

for song_batch, genre_batch in dataset.take(1):
    tf.gather(song_batch)
    notes = [tf.slice(song_batch, [0, 0, i], [-1, 0, i]) for i in range(8192)]
    print(notes[0])

2022-11-17 18:59:03.635505: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected size[2] in [0, 5], but got 6 [Op:Slice]